In [2]:
import numpy as np
import pickle as pkl
import os
from sklearn.preprocessing import RobustScaler

In [ ]:
a = np.loadtxt('../GermanCredit/prod/german.data', dtype='str')
new_data = np.zeros((1000,70))

In [ ]:
def foo(attr, start, count):
    return ['{}{}'.format(attr, i) for i in range(start, count+1)]

a1 = foo('A1',1,4)
a2 = [1]
a3 = foo('A3',0,4)
a4 = foo('A4',0,10)
a5 = [4]
a6 = foo('A6',1,5)
a7 = foo('A7',1,5)
a8 = [7]
a9 = foo('A9',1,5)
a10 = foo('A10',1,3)
a11 = [10]
a12 = foo('A12',1,4)
a13 = [12]
a14 = foo('A14',1,3)
a15 = foo('A15',1,3)
a16 = [15]
a17 = foo('A17',1,4)
a18 = [17]
a19 = foo('A19',1,2)
a20 = foo('A20',1,2)

headers = a1 +a2 +a3 +a4 +a5 +a6 +a7 +a8 +a9 +a10+a11+a12+a13+a14+a15+a16+a17+a18+a19+a20
len(headers)

In [ ]:
for i,h in enumerate(headers):
    if not isinstance(h, int):
        new_data[:,i][np.where(a == h)[0]] = 1.
    else:
        new_data[:,i] = a[:,h].astype(float)
        
# columns to be deleted        
del_list = []
for i in range(new_data.shape[1]):
    if np.sum(new_data[:,i]) == 0:
        del_list.append(i)
delete_cols =  set(list(range(33,38)) + del_list[:-1]) #delete the colums for gender and unused ones, keep one for rels
delete_cols

In [ ]:
headers_del = []
for i in delete_cols:
    if i<len(headers):
        headers_del.append(i)
        print(headers[i])
headers_ = []
for i,h in enumerate(headers):
    if i not in headers_del:
        headers_.append(headers[i])
headers_.insert(4,"A9_Gender")
headers_.append("RELEVANCE")

In [ ]:
headers_[4]

In [ ]:
males = np.concatenate((np.where(a == 'A91')[0], np.where(a == 'A93')[0], np.where(a == 'A94')[0]))
females = np.where(a == 'A92')[0]

In [ ]:
new_data_del = np.delete(new_data, list(delete_cols), 1)
#add gender in the 4th column
new_data_ins = np.insert(new_data_del,4,np.zeros(1000),axis=1)
new_data_ins[:,4][males]=1
#add relevances in the last column
new_data_ins[:,-1][a[:,-1].astype(float)==1]=1.0

In [3]:
data = np.loadtxt('../GermanCredit/prod/german_mod.data')
data_ = RobustScaler(with_centering=False).fit_transform(data)
group_column_number = 14
np.set_printoptions(suppress=True)

In [4]:
def get_nonzero(vals):
    return np.nonzero(vals)[0]

def get_train_test_splits(idxs):
    count_tr = int(len(idxs) * 0.35)
    count_va = int(len(idxs) * 0.35) + count_tr
    rand_idxs = np.random.rand(idxs.shape[0]).argsort()
    train_idxs = idxs[rand_idxs[:count_tr]]
    val_idxs = idxs[rand_idxs[count_tr:count_va]]
    test_idxs = idxs[rand_idxs[count_va:]]
    return train_idxs, val_idxs, test_idxs

In [5]:
def get_ranking_rel(t_p, t_n, rel_ratio, non_rel_ratio):
    data_rel = data_[np.random.choice(t_p, rel_ratio, replace=False)]
    data_non_rel = data_[np.random.choice(t_n, non_rel_ratio, replace=False)]
    ranking = np.vstack([data_rel, data_non_rel])
    rand_idxs = np.random.rand(ranking.shape[0]).argsort()
    return ranking[rand_idxs][:,:-1], ranking[rand_idxs][:,-1]

def get_final_rankings(num_queries_tr, num_queries_va, num_queries_te, rel_ratio, split, save=False):
    non_rel_ratio = 20 - 2*rel_ratio
    train_rankings = []
    train_rels = []
    test_rankings = []
    test_rels = []
    val_rankings = []
    val_rels = []
    
    rels = data[:,-1]
    
    if group_column_number:
        #TODO - deal with validation set
        gender = data[:,group_column_number]
        l = []
        for g in [0.,1.]:
            for r in [0.,1.]:
                l.append(get_nonzero(np.logical_and(gender==g, rels==r)))
        gen_f_rel_n, gen_f_rel_p, gen_m_rel_n, gen_m_rel_p = l

        tr_gen_f_rel_n, va_gen_f_rel_n, te_gen_f_rel_n = get_train_test_splits(gen_f_rel_n)
        tr_gen_f_rel_p, va_gen_f_rel_p, te_gen_f_rel_p = get_train_test_splits(gen_f_rel_p)
        tr_gen_m_rel_n, va_gen_m_rel_n, te_gen_m_rel_n = get_train_test_splits(gen_m_rel_n)
        tr_gen_m_rel_p, va_gen_m_rel_p, te_gen_m_rel_p = get_train_test_splits(gen_m_rel_p)

        tr_p, va_p, te_p = np.hstack([tr_gen_m_rel_p, tr_gen_f_rel_p]), np.hstack(
            [va_gen_f_rel_p, va_gen_m_rel_p]), np.hstack([te_gen_m_rel_p, te_gen_f_rel_p])
        tr_n, va_n, te_n = np.hstack([tr_gen_m_rel_n, tr_gen_f_rel_n]), np.hstack(
            [va_gen_f_rel_n, va_gen_m_rel_n]), np.hstack([te_gen_m_rel_n, te_gen_f_rel_n])
    else:
        data_p = np.nonzero(rels==1.)[0]
        data_n = np.nonzero(rels==0.)[0]

        tr_p, va_p, te_p = get_train_test_splits(data_p)
        tr_n, va_n, te_n = get_train_test_splits(data_n)
            
    for i in range(num_queries_tr):
        ranking_train, rel_train = get_ranking_rel(tr_p, tr_n, rel_ratio, non_rel_ratio)
        train_rankings.append(ranking_train)
        train_rels.append(rel_train)
    for i in range(num_queries_te):
        ranking_test, rel_test = get_ranking_rel(te_p, te_n, rel_ratio, non_rel_ratio)
        test_rankings.append(ranking_test)
        test_rels.append(rel_test)
    for i in range(num_queries_va):
        ranking_val, rel_val = get_ranking_rel(va_p, va_n, rel_ratio, non_rel_ratio)
        val_rankings.append(ranking_val)
        val_rels.append(rel_val)
        
    train_set = (train_rankings, train_rels)
    test_set = (test_rankings, test_rels)
    val_set = (val_rankings, val_rels)
    if save:        
        dir_name = "../GermanCredit/prod/35:35:30_group{}_double/split{}/".format(group_column_number, split)
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        base_str = dir_name + "relevance_german_mod_full"
        pkl.dump(train_set, open('{}_train_{}_{}:{}_split{}.pkl'.format(
            base_str,num_queries_tr, rel_ratio, non_rel_ratio,split), 'wb'))
        pkl.dump(test_set, open('{}_test_{}_{}:{}_split{}.pkl'.format(
            base_str, num_queries_te, rel_ratio, non_rel_ratio,split), 'wb'))
        pkl.dump(val_set, open('{}_val_{}_{}:{}_split{}.pkl'.format(
            base_str, num_queries_va, rel_ratio, non_rel_ratio, split), 'wb'))
    return train_set, test_set

In [6]:
for i in range(5):
    get_final_rankings(500, 500, 500, 1,i, save=True)